# 01. Importing Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02. Importing Data

In [2]:
# creating path folder as path
path = r'C:\Users\mojis\Documents\09-2022-Instacart Basket Analysis(Task 4 careerfoundary)\02 Data\Prepared Data(cleaned)'

In [3]:
# Importing orders_product_merge dataframe
ords_prods_merge = pd.read_pickle(r'C:\Users\mojis\Documents\09-2022-Instacart Basket Analysis(Task 4 careerfoundary)\02 Data\Prepared Data(cleaned)\orders_products_merged_2.pkl')

In [4]:
# checking dimension
ords_prods_merge.shape

(32404859, 18)

# 03. Aggregated mean

In [5]:
# Finding aggregated mean of the 'order_number' column group by 'department_id'
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


##### Question: How do the results for the entire dataframe differ from those of the subset?
###### Answers:  There are some slight difference in the mean output number.

### 04. Transform and loc() functions to create loyalty_flag column

In [6]:
#first step is to Aggregate.
# Aggregating order_number using maximum and grouping them by user_id
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
# using head functions to check the columns of the dataframe
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,_merge,price_range_loc,busiest_day,Busiest_days,Busiest_period_of_day,max_order
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,both,Mid-range product,Regularly busy,Regularly busy,Average orders,32
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,both,Mid-range product,Busiest day,Two busiest days,Average orders,5
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,both,Mid-range product,Regularly busy,Two slowest days,Most orders,3
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,both,Mid-range product,Least busy,Two slowest days,Average orders,3


### 4b.  Creating Loyalty Flag for existing customers

In [8]:
# Grouping max_order that are greater than 40 into 'Loyal customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
# Grouping max_order that <=40 and > 10 into 'Regular customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
# Grouping max_order that are <=10 into 'New customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
# Checking the frequency of the loyalty_flag column
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [12]:
# checking new dimension
ords_prods_merge.shape

(32404859, 20)

## 05. Difference between the spending habit of the 3 type of customers created in loyalty_flag column

In [13]:
# Finding the Basic Statistics of product prices for each loyalty category using aggregation and groupby funtion
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['min', 'max','sum', 'mean']})

prices                                 
                    min      max          sum       mean
loyalty_flag                                            
Loyal customer      1.0  99999.0  106814042.2  10.386336
New customer        1.0  99999.0   83011787.2  13.294670
Regular customer    1.0  99999.0  198391693.2  12.495717

### 06. Transform and loc() functions to create spender_flag column

In [14]:
#first step is to Aggregate.
# Aggregating prices using mean and grouping them by user_id
ords_prods_merge['Average_prices'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [15]:
# using head functions to check the new column created in the dataframe
ords_prods_merge[['user_id', 'prices','Average_prices']].head()

,user_id,prices,Average_prices
0,138,5.8,6.935811
1,138,5.8,6.935811
2,709,5.8,7.930208
3,764,5.8,4.972414
4,764,5.8,4.972414


In [16]:
# Checking new dimension
ords_prods_merge.shape

(32404859, 21)

### 6b. Creating spender_flag for each user based on the average prices

In [17]:
# flagging average_prices of products purchased by user which are lower than 10 as 'low spender'
ords_prods_merge.loc[ords_prods_merge['Average_prices'] <10, 'spender_flag'] = 'Low spender'

In [18]:
# flagging average_prices of products purchased by user which are higher than or equal 10 as 'High spender'
ords_prods_merge.loc[ords_prods_merge['Average_prices']>= 10, 'spender_flag'] = 'High spender'

In [19]:
# checking the frequency of spender_flag column
ords_prods_merge['spender_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spender_flag, dtype: int64

In [20]:
# Checking the new column
ords_prods_merge[['user_id','prices', 'Average_prices', 'spender_flag']].head()

,user_id,prices,Average_prices,spender_flag
0,138,5.8,6.935811,Low spender
1,138,5.8,6.935811,Low spender
2,709,5.8,7.930208,Low spender
3,764,5.8,4.972414,Low spender
4,764,5.8,4.972414,Low spender


### 07. Creating order frequency flag that marks the regularity of a user's ordering behavior according to the median in the 'days_since_prior_order'

In [21]:
# first step is to aggregate 
# creating median of 'days_since_prior_order' using transform() and grouping them by user_id
ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [22]:
# using head to check new column created
ords_prods_merge[['user_id','days_since_prior_order', 'median_days_since_prior_order']].head()

,user_id,days_since_prior_order,median_days_since_prior_order
0,138,3.0,8.0
1,138,20.0,8.0
2,709,6.0,8.0
3,764,NaN,9.0
4,764,9.0,9.0


### 7b.  secondly is using loc to create the order frequency flag

In [23]:
#If the median of “days_since_prior_order” is higher than 20, then customer are labeled a “Non-frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] >20, 'order_frequency_flag'] = 'Non-frequent customer'

In [24]:
# If the median is higher than 10 and lower than or equal to 20, then customer are labeled a “Regular customer.”
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] >10) & (ords_prods_merge['median_days_since_prior_order'] <= 20) , 'order_frequency_flag'] = 'Regular customer'

In [25]:
# If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <=10, 'order_frequency_flag'] = 'Frequent customer'

In [26]:
# Checking the frequency of the order_frequency_flag column
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: order_frequency_flag, dtype: int64

In [27]:
# checking new dimension
ords_prods_merge.shape

(32404859, 24)

In [28]:
# checking new column using head function
ords_prods_merge[['user_id', 'days_since_prior_order','user_id', 'median_days_since_prior_order','order_frequency_flag' ]].head()

,user_id,days_since_prior_order,user_id,median_days_since_prior_order,order_frequency_flag
0,138,3.0,138,8.0,Frequent customer
1,138,20.0,138,8.0,Frequent customer
2,709,6.0,709,8.0,Frequent customer
3,764,NaN,764,9.0,Frequent customer
4,764,9.0,764,9.0,Frequent customer


# cleaning process(dealing with outlier)

In [29]:
#Running checks
ords_prods_merge.loc[ords_prods_merge['prices'] > 100]

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,price_range_loc,busiest_day,Busiest_days,Busiest_period_of_day,max_order,loyalty_flag,Average_prices,spender_flag,median_days_since_prior_order,order_frequency_flag
13100147,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,912404,17,12,2,14,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
13100148,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,603376,17,22,6,16,...,High-range product,Regularly busy,Regularly busy,Most orders,40,Regular customer,108.648299,High spender,5.0,Frequent customer
13100149,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,3264360,135,2,2,21,...,High-range product,Regularly busy,Regularly busy,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
13100150,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,892534,135,3,0,8,...,High-range product,Busiest day,Two busiest days,Average orders,4,New customer,1154.792308,High spender,12.0,Regular customer
13100151,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,229704,342,8,1,19,...,High-range product,Regularly busy,Two busiest days,Average orders,16,Regular customer,114.426619,High spender,23.0,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21786876,33664,2 % Reduced Fat Milk,84,16,99999.0,2249946,204099,29,0,8,...,High-range product,Busiest day,Two busiest days,Average orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
21786877,33664,2 % Reduced Fat Milk,84,16,99999.0,2363282,204099,31,0,9,...,High-range product,Busiest day,Two busiest days,Most orders,39,Regular customer,1106.743956,High spender,4.0,Frequent customer
21786878,33664,2 % Reduced Fat Milk,84,16,99999.0,3181945,204395,13,3,15,...,High-range product,Regularly busy,Two slowest days,Most orders,15,Regular customer,451.153540,High spender,5.0,Frequent customer
21786879,33664,2 % Reduced Fat Milk,84,16,99999.0,2486215,205227,7,3,20,...,High-range product,Regularly busy,Two slowest days,Average orders,12,Regular customer,1178.381871,High spender,12.0,Regular customer


In [30]:
#changing all prices greater than 100 with NaNs
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [31]:
# checking max
ords_prods_merge['prices'].max()

25.0

 # 09. Exporting as pickle file

In [32]:
# exporting ords_prods_merge as orders_products_merge_recent.pkl
ords_prods_merge.to_pickle(r'C:\Users\mojis\Documents\09-2022-Instacart Basket Analysis(Task 4 careerfoundary)\02 Data\Prepared Data(cleaned)\orders_products_merge_3.pkl')